In [3]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from sklearn.model_selection import KFold


In [4]:
# データを格納するリスト
X = []
Z = []

# 画像のサイズ
IMG_SIZE = 150

# 各花のディレクトリのパス
FLOWER_DAISY_DIR = '../input/flowers/daisy'
FLOWER_DANDI_DIR = '../input/flowers/dandelion'
FLOWER_ROSE_DIR = '../input/flowers/rose'
FLOWER_SUNFLOWER_DIR = '../input/flowers/sunflower'
FLOWER_TULIP_DIR = '../input/flowers/tulip'

# フラグを初期化
added_flowers = set()


In [5]:
# データの格納とコメントを追加した関数
def make_train_data(flower_type, DIR):
    if flower_type not in added_flowers:
        added_flowers.add(flower_type)
        # 各花のデータを格納するためのループ
        for img in tqdm(os.listdir(DIR)):
            # ラベルを花の名前に設定
            label = str(flower_type)
            # 画像ファイルのパスを生成
            path = os.path.join(DIR, img)
            # 画像をカラーで読み込み
            img = cv2.imread(path, cv2.IMREAD_COLOR)
            # 画像サイズをリサイズ
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

            # データをリストに追加
            X.append(np.array(img))
            Z.append(str(label))


In [6]:
# 各花に対してデータを作成
make_train_data('Daisy', FLOWER_DAISY_DIR)
make_train_data('Dandelion', FLOWER_DANDI_DIR)
make_train_data('Rose', FLOWER_ROSE_DIR)
make_train_data('Sunflower', FLOWER_SUNFLOWER_DIR)
make_train_data('Tulip', FLOWER_TULIP_DIR)


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 984/984 [00:09<00:00, 106.16it/s]


In [16]:
# 1. データの前処理
le = LabelEncoder()
Y = le.fit_transform(Z)
Y = torch.tensor(Y, dtype=torch.long)
X = np.array(X)
X = X / 255.0  # ピクセル値を正規化
X = torch.tensor(X, dtype=torch.float32)
X = X.permute(0, 3, 1, 2)  # PyTorchはチャネルを先に要求


In [17]:
# データを訓練セットと検証セットに分割
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

# データローダーの作成
train_dataset = TensorDataset(X_train, Y_train)
val_dataset = TensorDataset(X_val, Y_val)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [11]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # 最初の畳み込み層：32フィルタ、5x5カーネル、ReLU活性化関数、入力チャネルは3（RGB）
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, padding='same')
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # 二番目の畳み込み層：64フィルタ、3x3カーネル、ReLU活性化関数
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding='same')
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # 三番目の畳み込み層：96フィルタ、3x3カーネル、ReLU活性化関数
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=96, kernel_size=3, padding='same')
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        # 四番目の畳み込み層：96フィルタ、3x3カーネル、ReLU活性化関数
        self.conv4 = nn.Conv2d(in_channels=96, out_channels=96, kernel_size=3, padding='same')
        self.relu4 = nn.ReLU()
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Flatten層
        self.flatten = nn.Flatten()

        # 全結合層
        self.fc1 = nn.Linear(96 * 9 * 9, 512)  # 96フィルタ、9x9サイズ（入力サイズに依存）
        self.relu5 = nn.ReLU()

        # 出力層：5クラスに分類するための層
        self.fc2 = nn.Linear(512, 5)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = self.pool3(self.relu3(self.conv3(x)))
        x = self.pool4(self.relu4(self.conv4(x)))
        x = self.flatten(x)
        x = self.relu5(self.fc1(x))
        x = self.fc2(x)
        return x


In [18]:
# 2. モデルの構築
model = SimpleCNN()


In [19]:
# 3. 訓練と検証
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)


In [22]:
# 訓練関数
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    for epoch in range(num_epochs):
        model.train()
        total_loss, total_correct = 0, 0
        for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}', leave=False):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_correct += (predicted == labels).sum().item()

        scheduler.step(total_loss)

        print(f'Epoch {epoch+1}/{num_epochs}')
        print(f'Train loss: {total_loss / len(train_loader)}')
        print(f'Train accuracy: {total_correct / len(Y_train)}')


In [23]:
# 訓練の実行
train_model(model, criterion, optimizer, scheduler)


Epoch 1/25
Train loss: 1.1428918093442917
Train accuracy: 0.5207066319142775


Epoch 2/25
Train loss: 0.9601964006821314
Train accuracy: 0.6128004633651897


Epoch 3/25
Train loss: 0.8675583810717972
Train accuracy: 0.6594265855777585


Epoch 4/25
Train loss: 0.7975318335272648
Train accuracy: 0.6982334202143065


Epoch 5/25
Train loss: 0.6971795895585308
Train accuracy: 0.7355922386330727


...

Epoch 22/25
Train loss: 0.016500352475473105
Train accuracy: 0.9953663481030988


Epoch 23/25
Train loss: 0.011112295585108869
Train accuracy: 0.9979727772951057


Epoch 24/25
Train loss: 0.0058475763315881425
Train accuracy: 0.9985519837822183


Epoch 25/25
Train loss: 0.0038622906955694176
Train accuracy: 0.9991311902693311


: 